# Moonshot红利策略完结篇
\
我们已分4期进行解读了中金2023年12月的一份研报，名为《在手之鸟，红利优选策略》。而这是本系列最后一篇,运用之前的Moonshot回测框架，按照研报的逻辑完成最终红利策略的构建。


## 因子构建方式
红利策略的核心是股息率。事实上股息率因子最早的资产定价因子之一，在Harvey, Liu & Zhu (2016)等一系列经典论文中，Dividend Yield都是作为估值类因子的重要组成部分，但其显著性总体弱于其他经典因子如市值，P/M等。股息率常出现在现金流稳定的公司，是价值因子的一个侧面，此外从股息收益安全性和管理层信号理论信号理论而言，都可以做出恰当的经济学解释。
从股息收益、资本利得和风险规避三个角度，研报构建了多个因子，这些因子对于股票未来的分红有一定的预测效果。下面先简述因子构建方式，具体因子构建代码放在后文。

### 每期股息率排名前500（DP） 以及 2年平均股息率的因子标准分（DP2）
含义：高股息的股票未来股息率和每股分红也处于高位。\
计算：股息率的计算在前述系列文章中已有论证，从tushare的接口daily_basic取出股息率字段dv_ttm。2年平均股息率dividend_yield_2y_avg 由一年前的dv_ttm，一年前close，与当前的dv_ttm，close搭配计算。具体计算过程参见代码。

### 派息率（DPR）
含义：高派息率股票未来分红表现或较难持续，而派息率适中或有利于未来收益。具体来说，若派息率处于低位，或说明公司分红表现欠佳，组合整体收益表现一般；若派息率处于高位，分红行为或难以持续。\
计算：派息率是分红/净利润，也就是派息率 = 股息率 /  EP = 股息率 * PE，因此从daily_basic取出pe_ttm * dv_ttm。

### 市值筛选（总市值 > 50 亿元）
含义：小市值加剧组合整体波动表现。\
计算：tushare的接口daily_basic取出total_mv，单位为万元。

### 换手波动率
含义：换手率波动率在红利股票池中具有较好的选股能力，引入换手率波动率的低波因子有利于提高组合安全边际。\
计算：tushare的接口daily_basic 自由流通口径 turnover_rate_f 计算近一个月的标准差。

### EP 标准分
含义： 根据 EP 分组的估值均值回复较为显著,EP 最大组合估值最低。\
计算： daily_basic 接口中，可以直接使用PE来倒推EP，然后取5年的标准分。

### 较上一个月分红金额TTM增长
含义： 持续、稳定且具增长性的分红传递了管理层对现金流与盈利质量的信心，有利于长期股东回报与估值稳定。
计算：可用 dv_ttm 与 总市值计算dividend_ttm，随后对比月度变化

### 股东数量标准分
含义： 当股东数量降低，或是因为有信息优势的投资者吸筹看好未来公司业绩。\
计算： 通过股东户数接口stk_holdernumber获取历史序列计算。

### 审计意见（剔除非标准无保留）
含义： 非标准审计意见往往提示财务不确定性升高或潜在风险事件。\
计算与口径： 通过审计意见接口fina_audit读取audit_result。

### 经营现金流资产比（Operating Cash Flow / Total Assets）
含义： 若公司现金流充裕，则有利于维系未来的高分红水平。但若公司现金或现金流紧缺依然选择现金分红，该行为或不可持续，且具有一定风险。\
计算： 取 n_cashflow_act，可通过 cashflow_vip, 与同期 total_assets，使用 balancesheet_vip, 并计算n_cashflow_act / total_assets。

### 留存收益资产比（Retained Earnings / Total Assets）
含义： 如果用留存收益比总资产来衡量生命周期，在成熟期留存收益资产占比较高，发现股利支付集中在这样的公司。 \
计算： balancesheet_vip 取 undistr_porfit 和 total_assets 字段 ，RETA = undistr_porfit / total_assets。

### 净利润业绩稳健（np_std）
含义：当期财务收益稳健因子（如过去八期的净利润标准分）较大时，红利股票未来的 ROE 或维持较高水平，因此财务收益稳健因子对公司未来盈利具有一定的预测能力\
计算： income_vip接口取n_income_attr_p，计算过去八期的净利润标准分。

## 回测结果
按照研报的思路进行回测，我们使用系列的moonshot完成回测。

In [30]:
import sys
sys.path.append(str(Path(".").parent))

import datetime
from store import Bars, ParquetStorage, CalendarModel
from fetchers import (fetch_bars_ext, fetch_fina_audit, fetch_dividend)

from moonshot import Moonshot

data_home = Path("~/workspace/data").expanduser()

start = datetime.date(2023, 10, 30)
end = datetime.date(2023, 11, 30)

# 日历
calendar= CalendarModel(data_home/"rw/calendar.parquet")

bars = Bars()
bars.connect(data_home / "rw/bars.parquet")

barss = bars.get_bars_in_range(start, end)
barss.tail()


2025-11-08 17:50:20.187 | INFO     | store:load:53 - Calendar 将从 /Users/aaronyang/workspace/data/rw/calendar.parquet处加载数据
2025-11-08 17:50:20.252 | INFO     | store:load:53 - Calendar 将从 /Users/aaronyang/workspace/data/rw/calendar.parquet处加载数据


,date,asset,open,high,low,close,volume,amount,adjust
127298,2023-11-30,000524.SZ,8.61,8.95,8.61,8.95,148975.56,132050.676,4.927
127299,2023-11-30,000523.SZ,3.27,3.30,3.24,3.28,83177.80,27160.281,6.687
127300,2023-11-30,000521.SZ,6.11,6.13,6.04,6.08,88248.00,53656.617,5.427
127301,2023-11-30,000519.SZ,15.20,15.20,14.76,14.86,190493.11,284398.044,7.971
127302,2023-11-30,302132.SZ,44.71,44.93,43.43,43.86,79143.64,349037.982,5.976


## 基本财务数据

策略要求使用股息率、市值数据、市盈率、换手率等数据，这些数据来源于 tushare 的 daily_basic 接口。下面，我们就先获取这部分数据。

ms = Moonshot(barss)
# ADD FACTORS
daily_basic_after = pd.read_parquet('data_part2/daily_after.parquet')
ms.append_factor(daily_basic_after, "total_mv", resample_method="last")
ms.append_factor(daily_basic_after, "dv_ttm", resample_method="last")
ms.append_factor(daily_basic_after, "dividend_yield_2y_avg", resample_method="last")
ms.append_factor(daily_basic_after, "dividend_yield_2y_avg_noffill", resample_method="last")

ms.append_factor(daily_basic_after, "DPR", resample_method="last")
ms.append_factor(daily_basic_after, "turnover_rate_f_std", resample_method="last")
ms.append_factor(daily_basic_after, "inv_pe_ttm_zscore_5y", resample_method="last")
ms.append_factor(daily_basic_after, "dividend_ttm_increase_1M", resample_method="last")

# ADD HOLDERS
holder = pd.read_parquet('data_part2/holder_zscore_4y.parquet')
holder.rename(columns={"ts_code": "asset"}, inplace=True)
ms.append_factor(holder, "holder_z_score", resample_method="last")

# ADD AUDIT
final_audit_df = pd.read_parquet('data_part2/audit_reserve.parquet')
final_audit_df.rename(columns={"ts_code": "asset"}, inplace=True)
ms.append_factor(final_audit_df, "has_audit_reserve", resample_method="last")

# ADD CASHFLOW, INCOME, BALANCE SHEET
n_cashflow_act =pd.read_parquet('data_part2/n_cashflow_act.parquet')
ms.append_factor(n_cashflow_act, "n_cashflow_act", resample_method="last")
income_8q_zscore = pd.read_parquet('data_part2/income_8q_zscore.parquet')
income_8q_zscore.rename(columns={"ts_code": "asset"}, inplace=True)
ms.append_factor(income_8q_zscore, "profit_z_score", resample_method="last") 
balancesheet_asset_profit = pd.read_parquet('data_part2/assets_undistr_profit.parquet')
ms.append_factor(balancesheet_asset_profit, "undistr_porfit", resample_method="last") 
ms.append_factor(balancesheet_asset_profit, "total_assets", resample_method="last") 

In [31]:
from fetchers import fetch_daily_basic
basic_store_path = Path(data_home / "rw/daily_basic.parquet")

basic = ParquetStorage(basic_store_path, calendar, fetch_daily_basic)
basics = basic.get_and_fetch(start, end)

basics


,asset,date,close,turnover_rate,turnover_rate_f,dv_ratio,dv_ttm,pe,pe_ttm,total_mv,circ_mv
0,000761.SZ,2023-10-30,3.90,0.1922,1.2698,15.3845,NaN,NaN,NaN,1.602205e+06,1.446205e+06
1,002159.SZ,2023-10-30,14.16,2.0792,3.0997,0.0000,NaN,NaN,86.5038,2.510587e+05,1.954490e+05
2,001270.SZ,2023-10-30,90.01,9.7491,9.7491,0.0000,0.2857,106.1398,112.8308,1.409000e+06,6.413693e+05
3,688435.SH,2023-10-30,61.60,11.9731,11.9731,NaN,NaN,139.0054,146.2263,5.143600e+05,1.134219e+05
4,002959.SZ,2023-10-30,55.49,2.7607,9.4694,1.0798,1.4275,22.4910,18.8294,8.688471e+05,8.665388e+05
...,...,...,...,...,...,...,...,...,...,...,...
127298,301387.SZ,2023-11-30,58.71,5.2233,5.2233,NaN,1.1923,39.2392,41.1937,4.461960e+05,1.032539e+05
127299,300122.SZ,2023-11-30,65.22,0.8732,1.1337,0.6133,0.5111,20.7624,18.4974,1.565280e+07,9.264621e+06
127300,300276.SZ,2023-11-30,4.28,10.2039,12.7506,0.0000,NaN,NaN,NaN,5.996464e+05,4.247923e+05
127301,600259.SH,2023-11-30,34.07,0.6728,1.1078,0.0000,NaN,49.3407,61.5218,1.146237e+06,1.122572e+06


我们把这些因子加入到 Moonshot 中。

首先是总市值。策略要求只把总市值大于50亿的个股加入股票池，这相当于 total_mv > 50000，因为 total_mv 的单位是万元。我们通过下面的方法来实现：

In [32]:
ms = Moonshot(barss)

transform = lambda x: 1 if x["total_mv"] > 50000 else 0
ms.append_factor(basics, "total_mv", resample="last", transform=transform)

通过一个 transform，我们就把这个因子转换成为交易信号。

接下来我们加入股息率排名因子。

In [33]:
def dv_rank_less_500(factor: pd.DataFrame):
    rank = factor.rank(method="first", ascending=False)
    factor[rank <= 500] = 1
    factor[rank > 500] = 0

    return factor

ms.append_factor(basics, "dv_ttm", transform=dv_rank_less_500, resample="last")

接下来，我们加入连续两年分红筛选。

我们先通过 `dividend`来获取分红数据。

In [34]:
dividend_store = ParquetStorage(data_home/"rw/dividend.parquet", 
                                calendar, 
                                fetch_dividend)

dividend_store.fetch(start, end, call_direct=True)

dividend = dividend_store.get()
dividend.head()


2025-11-08 17:50:21.406 | INFO     | tushare:_connect:29 - Connected to server tushare:5290


,asset,div_proc,stk_div,stk_bo_rate,stk_co_rate,cash_div,cash_div_tax,record_date,ex_date,pay_date,div_listdate,imp_ann_date,date,fiscal_year
0,300708.SZ,实施,0.0,NaN,NaN,0.16,0.16,20240227,20240228,20240228,None,20240221,2024-01-27,2023
1,300803.SZ,股东大会通过,0.0,NaN,NaN,0.00,0.00,None,None,None,None,None,2024-01-27,2023
2,002286.SZ,实施,0.0,NaN,NaN,0.08,0.08,20240307,20240308,20240308,None,20240301,2024-02-07,2023
3,300758.SZ,股东大会通过,0.0,NaN,NaN,0.00,0.00,None,None,None,None,None,2024-02-07,2023
4,835438.BJ,股东大会通过,0.0,NaN,NaN,0.00,0.00,None,None,None,None,None,2024-02-20,2023


计算是否连续两年的过程比较复杂，我们需要将它封装成一个函数，直接把信号加入到策略中。

In [35]:
import pandas as pd
def pre_process(df):
    df["month"] = pd.to_datetime(dividend["date"]).dt.to_period("M")

    # 某些个股在一个财年，可能会有多次分红，我们取最早的一次
    (
        df.sort_values(["asset", "fiscal_year", "date"])
        .groupby(["asset", "fiscal_year"], as_index=False)
        .first()
    )

    cols = ["asset", "month", "fiscal_year"]
    df = df[cols].set_index(["asset", "month"])
    
    all_assets = df.index.levels[0].unique()
    all_dates = df.index.levels[1].unique()

    start, end = min(all_dates), max(all_dates)

    all_months = pd.period_range(start, end, freq="M")

    # 生成所有可能的（月份，股票）组合作为基础索引
    index = pd.MultiIndex.from_product([all_assets, all_months], names=["asset", "month"])
    # 生成所有可能的（月份，股票）组合作为基础索引
    index = pd.MultiIndex.from_product([all_assets, all_months], names=["asset", "month"])

    expanded = pd.merge(pd.DataFrame(index=index), 
                  df, how="left", 
                  left_index=True, 
                  right_index=True
            )

    # 将 fiscal_year 前向填充
    expanded = expanded.groupby(level = "asset").ffill()

    return expanded

def calc_asset_flag(group: pd.DataFrame):
    df = group.droplevel(level=0)

    df["month_num"] = df.index.month
    df["year"] = df.index.year

    # 这里无法使用 rolling，因为 rolling 后面要跟聚合函数，不能返回 set
    df["prev_fiscal_set"] = [
        set(x) for x in df["fiscal_year"].rolling(24)
    ]

    def calc_row_flag(row):
        prev_fiscal = row["prev_fiscal_set"]
        month_num = row["month_num"]
        year = row["year"]

        if month_num > 4:  # 4 月之后，必须最近两年财年都有分红
            flag = ((year - 1) in prev_fiscal) and ((year - 2) in prev_fiscal)
            return flag
        else:  # 4 月之前，最近两财年有分红，或者之前两个财年有分红。
            flag = (((year - 1) in prev_fiscal) and ((year - 2) in prev_fiscal)) or (
                ((year - 2) in prev_fiscal) and ((year - 3) in prev_fiscal)
            )
            return flag

    df["consective_div"] = df.apply(calc_row_flag, axis=1)

    return df.drop(columns=["month_num", "year", "prev_fiscal_set"])

def calc_consecutive_dividend_flag(dividend):
    preprocessed = pre_process(dividend)
    df = preprocessed.groupby(level="asset").apply(calc_asset_flag)
    # 去重
    return df[~df.index.duplicated(keep='first')].swaplevel()

dividend_yield_2y = calc_consecutive_dividend_flag(dividend)
dividend_yield_2y

,,fiscal_year,consective_div
month,asset,,
2024-01,000001.SZ,NaN,False
2024-02,000001.SZ,NaN,False
2024-03,000001.SZ,2023.0,False
2024-04,000001.SZ,2023.0,False
2024-05,000001.SZ,2023.0,False
...,...,...,...
2024-11,920992.BJ,2023.0,False
2024-12,920992.BJ,2023.0,False
2025-01,920992.BJ,2023.0,False


In [36]:
ms.append_factor(dividend_yield_2y, "consective_div")
returns, benchmark = ms.run()
ms.report(returns, benchmark, 'metrics')

KeyError: "Cannot interpret 'total_mv' as period"

接下来我们加入审计无保留意见的筛选。这项数据需要通过`pro.audit`来获取。

In [29]:
start = datetime.date(2018, 1, 1)
end = datetime.date(2024, 12, 31)

audit_store = ParquetStorage(data_home/"rw/audit.parquet", calendar, fetch_data_func = fetch_fina_audit)

audit_store.fetch(start, end, call_direct=True)

2025-11-08 16:49:41.226 | INFO     | tushare:_connect:29 - Connected to server tushare:5290
2025-11-08 17:32:12.129 | ERROR    | tushare:_receive_data:125 - Error receiving data: timed out
2025-11-08 17:32:12.134 | WARNING  | tushare:_send_request:78 - Connection failed (attempt 1/4): timed out
2025-11-08 17:32:12.136 | INFO     | tushare:_send_request:79 - Retrying in 1 seconds...
2025-11-08 17:32:12.137 | ERROR    | fetchers:fetch_fina_audit:132 - 获取 300051.SZ 的财务审计意见失败: name 'time' is not defined
2025-11-08 17:32:12.140 | INFO     | tushare:_connect:29 - Connected to server tushare:5290


KeyboardInterrupt: 

函数stock_pool_filter是针对股票的第一步筛选，划定了哪些股票可以进入股票池，共有4个条件。
1. 每期股息率排名前500。
2. 连续两年分红。转化为dividend_yield_2y_avg_noffill大于0，其中dividend_yield_2y_avg_noffill指的是dividend_yield_2y_avg计算完后不进行空值填充的结果，如果股票在这一个因子上为空，0，或负值，那显然连续不存在连续2年分红。
3. 过去十年没有审计意见。
4. 市值大于50亿。

In [ ]:
def stock_pool_filter(data: pd.DataFrame) -> pd.Series:
   
    df = data.copy()
    
    # 股息率前 500 名
    dv_rank = df.groupby(level="month")['dv_ttm'].transform(lambda x: x.rank(method='first', ascending=False))
    cond1 = dv_rank <= 500
    cond2 = df['dividend_yield_2y_avg_noffill'] > 0 # 连续两年有分红
    cond3 = df['has_audit_reserve'] == False # 无审计保留意见
    cond4 = df['total_mv'] >= 500000 # 总市值 > 50亿，总市值单位是 （万元）
    
    flag =  cond1 & cond2 & cond3 & cond4
    return flag.astype(int)

函数factor_screen描述核心的选股因子。按照研报的理解，对于在红利股票池有效性显著的指标，将使用指标的标准分；对于有效性偏低的股票，则构建阈值信号描述相关信息。
具体来说包括 4个标准分 和 6个信号事件
1. 两年股息率均值 dividend_yield_2y_avg	 越大越好	
2. 净利润业绩稳健 profit_z_score 	 越大越好
3. 股东数量变化 holder_z_score	 越小越好
4. 换手波动率 turnover_rate_f_std	 越小越好

5. 派息率前5分之一 +1   派息率后5分之一 -1， DPR
6. 经营现金流资产比后5分之一 -1  n_cashflow_act / total_assets	
7. 留存收益/资产比后5分之一 -1	undistr_porfit / total_assets
8. EP 前5分之一的股票+1，EP 后5分之一的股票 -1  inv_pe_ttm_zscore_5y
9. 分红预案日至股东大会公告日之间	由于无数据跳过
10. 最近1月分红TTM增长 +1	dividend_ttm_increase_1M

此外在回测之前还需要对全部数据进行填充。

In [ ]:
ms.data.sort_index(level=['month', 'asset'], inplace=True)

cols_to_ffill = ['total_mv', 'dv_ttm', 'dividend_yield_2y_avg', 'DPR', 'turnover_rate_f_std',
       'inv_pe_ttm_zscore_5y', 'dividend_ttm_increase_1M', 'holder_z_score',
       'has_audit_reserve', 'n_cashflow_act', 'profit_z_score',
       'undistr_porfit', 'total_assets']

# 在每个 asset 内按时间（month）前向填充
ms.data[cols_to_ffill] = (
    ms.data.groupby(level='asset')[cols_to_ffill]
           .ffill()
)

In [ ]:
def factor_screen(data: pd.DataFrame, top_n: int = 30) -> pd.Series:
    df = data.copy()

    # 只对上一层筛选通过的股票打分
    if 'flag' in df.columns:
        df = df[df['flag'] == 1].copy()

    factor_rank_info = {
        'dividend_yield_2y_avg': True,
        'profit_z_score': True,
        'holder_z_score': False,          # 越小越好
        'turnover_rate_f_std': False,     # 越小越好
    }

    def zscore_func(s, is_positive):
        s = s.copy()
        if not is_positive:
            s = -s
        mean = s.mean(skipna=True)
        std = s.std(skipna=True)
        if pd.isna(std) or std == 0:
            return pd.Series(0, index=s.index)
        z = (s - mean) / std
        return z.fillna(0)

    # 计算财务比率
    df['undistr_ratio'] = np.where(df['total_assets'] == 0, np.nan, df['undistr_porfit'] / df['total_assets'])
    df['cf_ratio'] = np.where(df['total_assets'] == 0, np.nan, df['n_cashflow_act'] / df['total_assets'])

    # 派息率前5分之一+1   派息率后5分之一	-1， DPR
    df['score_dpr'] = df.groupby('month')['DPR'].transform(
        lambda s: (
            (s >= s.quantile(0.8)).astype(int) -   # 前 20% → +1
            (s <= s.quantile(0.2)).astype(int)     # 后 20% → -1
        ).fillna(0)
    )

    # EP 前5分之一+1 EP 后5分之一-1 inv_pe_ttm_zscore_5y
    df['score_ep'] = df.groupby('month')['inv_pe_ttm_zscore_5y'].transform(
        lambda s: ((s >= s.quantile(0.8)).astype(int) - (s <= s.quantile(0.2)).astype(int)).fillna(0)
    )


    # 经营现金流资产比后5分之一	n_cashflow_act / total_assets	-1
    df['score_cf'] = df.groupby('month')['cf_ratio'].transform(
        lambda s: (-1 * (s <= s.quantile(0.2))).fillna(0).astype(int)
    )

    # 留存收益/资产比后5分之一 -1	undistr_porfit / total_assets
    df['score_undistr'] = df.groupby('month')['undistr_ratio'].transform(
        lambda s: (-1 * (s <= s.quantile(0.2))).fillna(0).astype(int)
    )

    # score_dividend_increase
    df['score_dividend_increase'] = df['dividend_ttm_increase_1M'].fillna(False).astype(int)

    
    for fac, is_positive in factor_rank_info.items():
        df[f"{fac}_score"] = df.groupby('month')[fac].transform(lambda s: zscore_func(s, is_positive))

    # 总分
    score_cols = [f"{fac}_score" for fac in factor_rank_info] + [
        'score_dpr', 'score_cf', 'score_undistr',
        'score_ep', 'score_dividend_increase']
    
    df['total_score'] = df[score_cols].sum(axis=1)

    # Top-N flag
    flag = df.groupby('month')['total_score'].transform(
        lambda s: (s.rank(method='first', ascending=False) <= top_n).astype(int)
    )

    # 对齐回原索引（未筛选股票置 0）
    flag = flag.reindex(data.index).fillna(0).astype(int)

    return flag

运行回测，并且按照研报的思路，引入中证红利指数作为对比。

In [ ]:
ms.screen(stock_pool_filter, data=ms.data)
ms.screen(factor_screen, data=ms.data, top_n=30)
print((ms.data['flag'] == 1).sum() / len(ms.data))
ms.calculate_returns()
# ms.report()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


calculate_df = ms.data.copy()
# 策略收益
strategy_return = (
    calculate_df
    .groupby(level='month')
    .apply(lambda x: x.loc[x['flag'] == 1, 'monthly_return'].mean())
    .to_frame(name='strategy_return')
)

# 基准收益：全市场平均
benchmark_return = (
    calculate_df
    .groupby(level='month')['monthly_return']
    .mean()
    .to_frame(name='benchmark_return')
)

# 指数数据
df_index = pd.read_excel('moonshoot past1234/中证红利指数_20251023_232411.xlsx')
df_index.rename(columns={'Date':'trade_date', 'Index':'close'}, inplace=True)
df_index['trade_date'] = pd.to_datetime(df_index['trade_date'])
df_index = df_index.sort_values('trade_date')

# 每月最后一个交易日
monthly_last = df_index.resample('M', on='trade_date').last()
monthly_last.index = monthly_last.index.to_period('M')
monthly_last = monthly_last[monthly_last.index >= '2018-01']
monthly_last['index_return'] = monthly_last['close'].pct_change().fillna(0)

# 合并所有收益
returns_df = strategy_return.join(benchmark_return)
returns_df = returns_df.join(monthly_last['index_return'])

# 计算累计收益
returns_df['strategy_cum'] = (1 + returns_df['strategy_return']).cumprod()
returns_df['benchmark_cum'] = (1 + returns_df['benchmark_return']).cumprod()
returns_df['index_cum'] = (1 + returns_df['index_return']).cumprod()

# 转为时间戳方便绘图
returns_df.index = returns_df.index.to_timestamp()

# 绘图
plt.figure(figsize=(12,6))
plt.plot(returns_df.index, returns_df['strategy_cum'], label='Strategy')
plt.plot(returns_df.index, returns_df['benchmark_cum'], label='EqualWeight Baseline', linestyle='--')
plt.plot(returns_df.index, returns_df['index_cum'], label='CSI Div Index', linestyle=':')
plt.xticks(rotation=45)
plt.xlabel('Month')
plt.ylabel('Cumulative Return')
plt.title('Strategy vs Benchmark vs Index')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# 总结
本次复现并没有完全重现研报的结果，主要原因可能在于研报对于基本面数据的细节处理更为精细化。但是总体来说，复现展示了红利因子的有效性，尤其是在2023年之后红利因子无论是单因子测试还是整体回测，都有较好的表现。对于单个因子的细节测试，读者可以自行对factor_screen的score_cols修改，从而测试选定因子的收益。


附录： 具体因子的处理代码